In [227]:
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.metrics import RMSE
from cornac.models import  ItemKNN, UserKNN, NMF, BPR, MF,VAECF
# from MF import MF
import pandas as pd
import numpy as np
import cornac
import math
# import seaborn as sns
# import matplotlib.pyplot as plt

In [228]:
rating_data_pd = pd.read_csv(
    "../data/ml-100K/indexed_interactions.csv",
    sep="\t",
    header=0,
    names=["userID", "itemID", "Rating", "Timestamp"],
)
rating_data = rating_data_pd.to_numpy()
rating_data.__len__()
rating_data_pd

,userID,itemID,Rating,Timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596
...,...,...,...,...
99273,875,173,3,880175444
99274,708,247,5,879795543
99275,37,981,1,874795795
99276,58,442,2,882399156


In [229]:

movie = '../data/ml-100K/i_id_mapping_genre.csv'

df_movie = pd.read_csv(movie, sep='\t')
print(f'shape: {df_movie.shape}')


df_movie.columns = ["item_id", "Name","genres","itemID"]
df_movie[:4]
movies = df_movie.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1
movies


movies

shape: (1348, 4)


,item_id,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,242,Kolya (1996),Comedy,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,302,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,377,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,51,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,346,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,1335,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,1251,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,1233,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,1380,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [230]:
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
movies

,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,Kolya (1996),Comedy,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [231]:
users = pd.read_csv("../data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
# users


users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
# user_features_numpy = users.to_numpy()
users

,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
195,49,0,writer,55105,0,20
185,39,1,executive,00000,1,6
21,25,0,writer,40206,2,20
243,28,0,technician,80525,3,19
165,47,0,educator,55113,4,3
...,...,...,...,...,...,...
938,26,1,student,33319,938,18
935,24,0,other,32789,939,13
929,28,1,scientist,07310,940,17
919,30,1,artist,90008,941,1


In [244]:
dataset = rating_data
dataset

array([[        0,         0,         3, 881250949],
       [        1,         1,         3, 891717742],
       [        2,         2,         1, 878887116],
       ...,
       [       37,       981,         1, 874795795],
       [       58,       442,         2, 882399156],
       [      304,       504,         3, 879959583]])

In [329]:

ratio_split = StratifiedSplit(
    data=dataset,test_size=0.2, rating_threshold=0.0, seed=123, verbose=True, chrono=True
)

hr_10 = cornac.metrics.HitRatio(k=20)
ndcg_10 = cornac.metrics.NDCG(k=20)
recall_10 = cornac.metrics.Recall(k=20)
prec_10 = cornac.metrics.Precision(k=20)
auc = cornac.metrics.AUC()
map = cornac.metrics.MAP()


# class cornac.models.vaecf.recom_vaecf.
# VAECF(name='VAECF', k=10, autoencoder_structure=[20], act_fn='tanh', 
# likelihood='mult', n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, trainable=True, verbose=False, seed=None, use_gpu=False)[source]

model=VAECF(k=10, autoencoder_structure=[20], name="vae100",act_fn="tanh", likelihood="mult", n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, seed=123, verbose=True)

models = [model]
cornac.Experiment(
    ratio_split, models=models, metrics=[hr_10, ndcg_10, recall_10,auc, prec_10,map]
).run()

rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1348
Number of ratings = 79039
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 943
Number of items = 1348
Number of ratings = 20239
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1348

[vae100] Training started!


100%|██████████| 100/100 [00:02<00:00, 42.94it/s, loss=6.84]



[vae100] Evaluation started!


Ranking: 100%|██████████| 943/943 [00:00<00:00, 2047.74it/s]


TEST:
...
       |    AUC | HitRatio@20 |    MAP | NDCG@20 | Precision@20 | Recall@20 | Train (s) | Test (s)
------ + ------ + ----------- + ------ + ------- + ------------ + --------- + --------- + --------
vae100 | 0.8483 |      0.7709 | 0.1295 |  0.1794 |       0.1280 |    0.1669 |    2.3355 |   0.4630



In [330]:
models[0].recommend(1)[:10]

[402, 140, 95, 23, 136, 52, 364, 25, 60, 278]

In [331]:
models[0].recommend(0)[:10]

[289, 49, 357, 157, 189, 489, 645, 89, 60, 52]

In [332]:
user_ids = users["userID"].to_numpy()
item_ids = movies["itemID"].to_numpy()
user_ids.__len__()

943

In [349]:
# get the top_k ratings for all users:
top_k = 20
reco_matrix = np.zeros((len(models), len(user_ids), top_k), dtype=int)
reco_matrix_mapped_items = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=int
)
reco_matrix_mapped_scores = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=float
)
reco_matrix_all = np.zeros((len(models), len(user_ids), len(item_ids)), dtype=int)


for u in user_ids:
    for i in range(len(models)):
        reco_items = models[i].recommend(u)
        items_mapped, mapped_scores = models[i].rank(
            user_idx=u, item_indices=list(item_ids)
        )
        reco_matrix_mapped_items[i][u] = items_mapped
        reco_matrix_mapped_scores[i][u] = mapped_scores
        reco_matrix_all[i][u] = reco_items
        reco_matrix[i][u] = reco_items[:top_k]

        # print(reco_matrix[0][3])

In [334]:
test_set_data = pd.DataFrame(ratio_split.test_set.uir_tuple).transpose()
test_set_data.columns = ["uid", "iid", "rating"]
test_set_data = test_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

test_set_data["uid"] = test_set_data["uid"].map(r_global_uid_map)
test_set_data["iid"] = test_set_data["iid"].map(r_global_iid_map)

In [335]:
train_set_data = pd.DataFrame(ratio_split.train_set.uir_tuple).transpose()
train_set_data.columns = ["uid", "iid", "rating"]
train_set_data = train_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

train_set_data["uid"] = train_set_data["uid"].map(r_global_uid_map)
train_set_data["iid"] = train_set_data["iid"].map(r_global_iid_map)

In [14]:
test_set_data.to_csv("testing_set_seed123_ml100k.csv", index=False, header=False)


In [15]:
train_set_data.to_csv("training_set_seed123_ml100k.csv", index=False, header=False)


In [355]:
np.save("reco_matrix_vaecf_100k",reco_matrix)


In [359]:
user_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [350]:
from collections import OrderedDict
# uid_map (OrderDict, required) – The dictionary containing mapping from user original ids to mapped integer indices.

# iid_map (OrderDict, required) – The dictionary containing mapping from item original ids to mapped integer indices.
sorted_by_values = OrderedDict(sorted(models[0].iid_map.items(), key=lambda item: item[1]))
keys_sorted_by_values = list(sorted_by_values.keys())

reco_items_scores_all = [OrderedDict() for _ in range(len(user_ids))]

for u in user_ids:
    actual_index_u = u
    mapped_index_u = models[0].uid_map[actual_index_u]
    # print(f"u {u} mapped u {mapped_index_u}")
    
    # for i in item_ids:
    mapped_scores = reco_matrix_mapped_scores[0][mapped_index_u]
    ordered_dict = OrderedDict(zip(keys_sorted_by_values, mapped_scores))
    reco_items_scores_all[actual_index_u] = ordered_dict
    
    
    

In [345]:
models[0].uid_map.items()

odict_items([(101, 0), (845, 1), (705, 2), (20, 3), (635, 4), (593, 5), (153, 6), (37, 7), (915, 8), (522, 9), (810, 10), (18, 11), (34, 12), (809, 13), (239, 14), (749, 15), (889, 16), (928, 17), (542, 18), (501, 19), (342, 20), (175, 21), (116, 22), (144, 23), (618, 24), (118, 25), (860, 26), (335, 27), (263, 28), (825, 29), (405, 30), (942, 31), (867, 32), (286, 33), (393, 34), (224, 35), (146, 36), (792, 37), (416, 38), (652, 39), (147, 40), (368, 41), (773, 42), (727, 43), (194, 44), (246, 45), (75, 46), (446, 47), (76, 48), (461, 49), (793, 50), (912, 51), (110, 52), (504, 53), (84, 54), (627, 55), (621, 56), (42, 57), (594, 58), (728, 59), (301, 60), (214, 61), (668, 62), (937, 63), (484, 64), (274, 65), (685, 66), (526, 67), (776, 68), (121, 69), (139, 70), (108, 71), (679, 72), (207, 73), (111, 74), (65, 75), (587, 76), (143, 77), (322, 78), (392, 79), (770, 80), (299, 81), (838, 82), (690, 83), (630, 84), (27, 85), (829, 86), (11, 87), (676, 88), (26, 89), (380, 90), (490, 91

In [346]:
models[0].iid_map.items()

odict_items([(177, 0), (289, 1), (153, 2), (361, 3), (321, 4), (723, 5), (983, 6), (60, 7), (949, 8), (140, 9), (113, 10), (136, 11), (1024, 12), (311, 13), (347, 14), (240, 15), (111, 16), (52, 17), (31, 18), (297, 19), (320, 20), (88, 21), (367, 22), (489, 23), (490, 24), (568, 25), (355, 26), (985, 27), (68, 28), (402, 29), (95, 30), (710, 31), (203, 32), (175, 33), (426, 34), (161, 35), (722, 36), (488, 37), (405, 38), (445, 39), (841, 40), (771, 41), (698, 42), (450, 43), (415, 44), (83, 45), (302, 46), (11, 47), (364, 48), (388, 49), (124, 50), (336, 51), (955, 52), (1008, 53), (265, 54), (1170, 55), (268, 56), (637, 57), (188, 58), (298, 59), (41, 60), (1037, 61), (1082, 62), (529, 63), (165, 64), (198, 65), (847, 66), (158, 67), (499, 68), (305, 69), (500, 70), (73, 71), (267, 72), (761, 73), (1066, 74), (1110, 75), (350, 76), (832, 77), (687, 78), (913, 79), (306, 80), (514, 81), (810, 82), (1010, 83), (1124, 84), (1122, 85), (412, 86), (644, 87), (562, 88), (1138, 89), (196, 

In [353]:
models[0].score(0)

array([2.3006564e-03, 8.2904669e-03, 3.1173679e-03, ..., 4.1458529e-06,
       5.3062736e-06, 3.5747630e-06], dtype=float32)

In [358]:
reco_items_scores_all.__len__()

943

In [354]:
import pickle
with open('score_dicts_vaecf100k.pkl', 'wb') as file:
    pickle.dump(reco_items_scores_all, file)

print("List of OrderedDicts saved to 'score_dicts.pkl'.")


List of OrderedDicts saved to 'score_dicts.pkl'.


In [356]:
reco_matrix[0][0]

array([289,  49, 357, 157, 189, 489, 645,  89,  60,  52,  95,   1, 133,
        31, 200, 360, 139, 148,  24, 174])